In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from datasets import load_dataset

# Load the dataset
squad = load_dataset("squad")

# see example of squad data
example = squad['train'][0]
for key in example:
    print(key, ":", example[key])

/home/anw270/mambaforge/envs/py311env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


id : 5733be284776f41900661182
title : University_of_Notre_Dame
context : Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
question : To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
answers : {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = 'bert-base-uncased'

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:

def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # truncate context, not the question
        max_length=500,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [5]:
tokenized_datasets = squad.map(prepare_train_features, batched=True, remove_columns=squad["train"].column_names)

In [6]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"finetune-BERT-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    dataloader_num_workers=2,
    dataloader_prefetch_factor=4
)

In [7]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()
     
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"].shuffle().select(range(2000)),
    eval_dataset=tokenized_datasets["validation"].shuffle().select(range(200)),
    data_collator=data_collator,
    tokenizer=tokenizer
)

/home/anw270/mambaforge/envs/py311env/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [8]:
# Run the trainer
trainer.train()

/home/anw270/mambaforge/envs/py311env/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,3.241447
2,No log,2.489918
3,No log,2.410760


TrainOutput(global_step=252, training_loss=3.0430799211774553, metrics={'train_runtime': 76.0787, 'train_samples_per_second': 78.866, 'train_steps_per_second': 3.312, 'total_flos': 1531035684000000.0, 'train_loss': 3.0430799211774553, 'epoch': 3.0})

In [9]:
instance = squad['train'][20]
context = instance['context']
question = instance['question']
print(context)
print(question)
print(instance['answers'])

All of Notre Dame's undergraduate students are a part of one of the five undergraduate colleges at the school or are in the First Year of Studies program. The First Year of Studies program was established in 1962 to guide incoming freshmen in their first year at the school before they have declared a major. Each student is given an academic advisor from the program who helps them to choose classes that give them exposure to any major in which they are interested. The program also includes a Learning Resource Center which provides time management, collaborative learning, and subject tutoring. This program has been recognized previously, by U.S. News & World Report, as outstanding.
What entity provides help with the management of time for new students at Notre Dame?
{'text': ['Learning Resource Center'], 'answer_start': [496]}


In [10]:
given_answer = instance['answers']['text'][0]  # Assuming the first answer is the correct one
given_answer_start = instance['answers']['answer_start'][0]
given_answer, given_answer_start

('Learning Resource Center', 496)

In [11]:
inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)
inputs = {k: v.to(device) for k, v in inputs.items()}
model.to(device)
# Get model's output
with torch.no_grad():
    output = model(**inputs)

In [12]:
# Get the predicted answer
start_idx = torch.argmax(output.start_logits)
end_idx = torch.argmax(output.end_logits)

predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1]))

In [13]:
predicted_answer, start_idx, end_idx, start_idx.item(), end_idx.item()

('learning resource center which provides time management, collaborative learning, and subject tutoring',
 tensor(111, device='cuda:0'),
 tensor(125, device='cuda:0'),
 111,
 125)

In [14]:
correct = (predicted_answer.lower() == given_answer.lower())
evaluation = 'Correct' if correct else f'Incorrect (Predicted: {predicted_answer}, Given: {given_answer})'

print(evaluation)

Incorrect (Predicted: learning resource center which provides time management, collaborative learning, and subject tutoring, Given: Learning Resource Center)


In [15]:


# Function to evaluate a single instance
def evaluate_instance(instance, device):
    context = instance['context']
    question = instance['question']
    given_answer = instance['answers']['text'][0]  # Assuming the first answer is the correct one

    # Tokenize the data
    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Apply the BERT model
    with torch.no_grad():  # No need to calculate gradients
        output = model(**inputs)

    # Get the predicted answer
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits)
    predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1]))

    return predicted_answer.lower() == given_answer.lower()

In [16]:
from tqdm import tqdm
# Evaluate the a number of instances
correct_count = 0
total_count = 100

for i in tqdm(range(total_count)):
    correct_count += evaluate_instance(squad['validation'][i], device)
# Calculate and output the accuracy
accuracy = correct_count / total_count
print(f'Accuracy: {accuracy * 100:.2f}%')

100%|██████████| 100/100 [00:00<00:00, 112.09it/s]

Accuracy: 24.00%
